### predict the chances for a goal in the next minute

In [1]:
cd ..

/Users/nellekesmits/Documents/projects/notebook.goalpredictor


In [2]:
# to do:
# read smoteENN functioning
# check anomaly detection effects
# what are the conclusions?

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statsbombpy import sb
import notebooks.utils as utils
import SoccermaticsForPython.FCPython as sfp_fcp
import SoccermaticsForPython as sfp
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_roc_curve
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [19]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN 
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import BalancedBaggingClassifier


#### import dataset

In [6]:
df = pd.read_csv('data/train_dataset.csv', index_col=0)
df.set_index('uniekid', inplace=True)

In [7]:
df.head(1)

,goal,competition_stage_Final,competition_stage_Group Stage,competition_stage_Quarter-finals,competition_stage_Round of 16,competition_stage_Semi-finals,under_pressure,aanvallende_helft,aanvallend_kwart,in_de_16,possession_team_Austria,possession_team_Belgium,possession_team_Croatia,possession_team_Czech Republic,possession_team_Denmark,possession_team_England,possession_team_Finland,possession_team_France,possession_team_Germany,possession_team_Hungary,possession_team_Italy,possession_team_Netherlands,possession_team_North Macedonia,possession_team_Poland,possession_team_Portugal,possession_team_Russia,possession_team_Scotland,possession_team_Slovakia,possession_team_Spain,possession_team_Sweden,possession_team_Switzerland,possession_team_Turkey,possession_team_Ukraine,possession_team_Wales,player_Aaron Ramsey,player_Adam Hložek,player_Adama Traoré Diarra,player_Admir Mehmedi,player_Adrien Rabiot,player_Albin Ekdal,player_Aleksandar Dragović,player_Aleksandar Trajkovski,player_Aleksandr Golovin,player_Aleksandr Sobolev,player_Aleksey Miranchuk,player_Alessandro Bastoni,player_Alessandro Florenzi,player_Alessandro Schöpf,player_Alex Král,player_Alexander Isak,player_Andrea Belotti,player_Andreas Christensen,player_Andreas Evald Cornelius,player_Andreas Skov Olsen,player_Andreas Ulmer,player_Andrej Kramarić,player_Andrew Robertson,player_Andrey Semenov,player_Andriy Yarmolenko,player_András Schäfer,player_André Miguel Valente Silva,player_Anga Dedryck Boyata,player_Ante Budimir,player_Ante Rebić,player_Antoine Griezmann,player_Anton Shunin,player_Antonio Rüdiger,player_Antonín Barák,player_Arijan Ademi,player_Artem Besedin,player_Artem Dovbyk,player_Artem Dzyuba,player_Attila Fiola,player_Attila Szalai,player_Axel Witsel,player_Aymeric Laporte,player_Bartosz Bereszyński,player_Ben Davies,player_Benjamin Pavard,player_Bernardo Mota Veiga de Carvalho e Silva,player_Billy Gilmour,player_Boban Nikolov,player_Borna Barišić,player_Breel-Donald Embolo,player_Bruno Miguel Borges Fernandes,player_Bruno Petković,player_Bryan Cristante,player_Bukayo Saka,player_Burak Yılmaz,player_Caglar Söyüncü,player_Callum William McGregor,player_Cengiz Umut Meraş,player_Cengiz Ünder,player_Che Adams,player_Chris Gunter,player_Christian Benteke Liolo,player_Christian Dannemann Eriksen,player_Christian Fassnacht,player_Christian Nørgaard,player_Christoph Baumgartner,player_Christopher James Mepham,player_Christopher Trimmel,player_Ciro Immobile,player_Clément Lenglet,player_Cody Mathès Gakpo,player_Connor Roberts,player_Corentin Tolisso,player_Cristiano Ronaldo dos Santos Aveiro,player_César Azpilicueta Tanco,player_Daler Kuzyaev,player_Daley Blind,player_Daniel Avramovski,player_Daniel Bachmann,player_Daniel James,"player_Daniel O""Shaughnessy",player_Daniel Olmo Carvajal,player_Daniel Wass,player_Danilo Luís Hélio Pereira,player_Danny Ward,player_Darko Churlinov,player_Darko Velkovski,player_David Brooks,player_David Marshall,player_David Olatukunbo Alaba,player_David Siger,player_Declan Rice,player_Dejan Kulusevski,player_Dejan Lovren,player_Denis Cheryshev,player_Denis Lemi Zakaria Lako Lado,player_Dennis Praet,player_Denzel Dumfries,player_Diogo José Teixeira da Silva,player_Djibril Sow,player_Dmitriy Barinov,player_Domagoj Vida,player_Domenico Berardi,player_Dominic Calvert-Lewin,player_Dominik Livaković,player_Donyell Malen,player_Dorukhan Toköz,player_Dries Mertens,player_Duje Ćaleta-Car,player_Dylan Levitt,player_Dávid Hancko,player_Eden Hazard,player_Eduard Sobol,player_Egzon Bejtulai,player_Eljif Elmas,player_Emerson Palmieri dos Santos,player_Emil Krafth,player_Emil Peter Forsberg,player_Emre Can,player_Endre Botka,player_Enis Bardhi,player_Eric García Martret,player_Ethan Ampadu,player_Evgeny Makarenko,player_Ezgjan Alioski,player_Fabian Lukas Schär,player_Fabián Ruiz Peña,player_Federico Bernardeschi,player_Federico Chiesa,player_Fedor Kudryashov,player_Ferhan Hasani,player_Ferrán Torres García,player_Filip Helander,player_Florian Grillits

#### create train and testset

In [8]:
# algemene train en testset
X = df.drop('goal', axis=1)
y = df['goal']

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)

In [9]:
y_test.value_counts(normalize=True)

0.0    0.982977
1.0    0.017023
Name: goal, dtype: float64

In [10]:
# verschillende balaning technieken testen met verschillende strategy's

In [11]:
# strategies = np.linspace(0,1,10)    #1 = 50/50 distribution between 0 and 1 class, 0.5 = 66 (0) and 33 (1) distribution
# strategy = 0.25
strategy = 0.5
# strategy = 0.75

In [12]:
rus = RandomUnderSampler(random_state=0,sampling_strategy=strategy)    # remove 0 class
ros = RandomOverSampler(random_state =0, sampling_strategy=strategy)   # add copies 1 class
smote = SMOTE(random_state =0,sampling_strategy=strategy)              # add synthetic 1 class
sme = SMOTEENN(random_state=0, sampling_strategy=strategy)             # https://towardsdatascience.com/imbalanced-classification-in-python-smote-enn-method-db5db06b8d50

X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
X_train_smoteENN, y_train_smoteENN = sme.fit_resample(X_train, y_train)

In [13]:
y_train_rus.value_counts(normalize=True)  

0.0    0.666667
1.0    0.333333
Name: goal, dtype: float64

In [14]:
y_train_ros.value_counts(normalize=True)  

0.0    0.666696
1.0    0.333304
Name: goal, dtype: float64

In [15]:
y_train_smote.value_counts(normalize=True)

0.0    0.666696
1.0    0.333304
Name: goal, dtype: float64

In [16]:
y_train_smoteENN.value_counts(normalize=True)  

0.0    0.559795
1.0    0.440205
Name: goal, dtype: float64

In [17]:
X_train_tot = [X_train, X_train_rus, X_train_ros, X_train_smote,X_train_smoteENN]
y_train_tot = [y_train, y_train_rus, y_train_ros, y_train_smote, y_train_smoteENN]
sampling_technique = ['nothing', 'undersampling' ,'oversampling' , 'smote' , 'smoteENN']

#### create model

In [18]:
models = []

models.append(("LogisticRegression",LogisticRegression(random_state=42)))
models.append(("SVC",SVC(random_state=42)))
# models.append(("LinearSVC",LinearSVC(random_state=42)))
models.append(("RandomForest",RandomForestClassifier(random_state=42)))
models.append(("bagging", BalancedBaggingClassifier(random_state=42)))
models.append(("bagging", BalancedRandomForestClassifier(max_depth=2, random_state=42)))

NameError: name 'BalancedRandomForestClassifier' is not defined

In [ ]:
results = []
techniques = []
names = []
train = []

for name, model in models:
    for technique, x, y in zip(sampling_technique, X_train_tot, y_train_tot):
        mod = make_pipeline(StandardScaler(), model)
        result = cross_val_score(mod, x, y,  cv=3)
        mod.fit(x, y)            
        names.append(name)
        results.append(result)
        techniques.append(technique)
        print(f'volgende ronde met een bepaalde sampling technique: {technique}:')
        print(name)
        print(result)
        print('with the following confusion matrix: ')
        plot_confusion_matrix(mod, X_test, y_test)  
        plt.show()

In [ ]:
for i in range(len(names)):
    print(names[i],results[i].mean(), techniques[i])

In [35]:
# one class classifier

In [ ]:
# random_forest = []
# for name, model in models:
#     if name == 'RandomForest':
#         random_forest.append(model)

In [ ]:
# #bagging technique

# balbag = BalancedBaggingClassifier(base_estimator = random_forest[0], n_estimators = 500, bootstrap = False,  bootstrap_features= True,
#                                   sampling_strategy = strategy, replacement = False, n_jobs = -1, random_state = 5)
# model_full_sample = balbag.fit(X_train, y_train)

In [ ]:
# plot_confusion_matrix(model_full_sample, X_test, y_test)  
# plt.show()

In [34]:
# utils.evaluate_model('Forest', model_full_sample, X_test, y_test)

# # rfc_single_disp = plot_roc_curve(model1, X_test, y_test)
# rfc_disp = plot_roc_curve(model_full_sample, X_test, y_test, ax=rfc_single_disp.ax_)
# rfc_disp.figure_.suptitle("ROC curve comparison")
# plt.show()

In [ ]:
# OUD

# mod.fit(X_train_rus, y_train_rus)
# plot_confusion_matrix(mod, X_test, y_test)  
# plt.show()

# scores = cross_val_score(mod, X_train_rus, y_train_rus, cv=5)
# print('scores: ',scores)

# mod.fit(X_train_ros, y_train_ros)
# plot_confusion_matrix(mod, X_test, y_test)  
# plt.show()

# scores = cross_val_score(mod, X_train_ros, y_train_ros, cv=5)
# print('scores: ',scores)

# mod.fit(X_train_smote, y_train_smote)
# plot_confusion_matrix(mod, X_test, y_test)  
# plt.show()

# scores = cross_val_score(mod, X_train_smote, y_train_smote, cv=5)
# print('scores: ',scores)

# forest = RandomForestClassifier(random_state=5, criterion = 'entropy', n_jobs = -1)
# model1 = forest.fit(X_train,y_train)
# plot_confusion_matrix(model1, X_test, y_test)  
# plt.show()

# scores = cross_val_score(model1, X_train, y_train, cv=5)
# print('scores: ',scores)